In [4]:
'''
@File        :alien_invasion.ipynb
@Description :
@Date        :2021/11/03 22:56:14
@Author      :Bo Sun
'''


import sys
import pygame
from settings import Settings
from ship import Ship
from bullet import Bullet

In [5]:
class AlienInvasion:
    """
    manage game resources and behaviors
    """

    def __init__(self) -> None:
        pygame.init()
        
        self.settings = Settings()

        self.screen = pygame.display.set_mode(
            (self.settings.screen_width, 
            self.settings.screen_height))
        pygame.display.set_caption("Alien Invasion")

        self.ship = Ship(self) # use self
        self.bullets = pygame.sprite.Group()

    def run_game(self):
        """
        start the main loop
        """
        while True:
            self._check_events()
            self.ship.update()
            self._update_bullets()
            self._update_screen()



    def _check_events(self):
        """
        monitor & react to keyborad and mouse events
        """
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit() # quit pygame
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                self._check_keydown_events(event)
            elif event.type == pygame.KEYUP:
                self._check_keyup_events(event)

    def _check_keydown_events(self, event):
        """
        response to keydown
        """
        if event.key == pygame.K_RIGHT:
            # move the ship right-ward
            self.ship.moving_right = True
        elif event.key == pygame.K_LEFT:
            # move the ship left-ward
            self.ship.moving_left = True
        elif event.key == pygame.K_q:
            # quit
            pygame.quit() # quit pygame
            sys.exit()
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()
    
    
    def _check_keyup_events(self, event):
        """
        response to keyup
        """
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False

    def _fire_bullet(self):
        """
        creat a bullet and add it to the bullets group
        """
        if len(self.bullets) < self.settings.bullets_allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)

    def _update_bullets(self):
        """
        update the bullet position 
        delete the vanishing bullet
        """
        # update the bullet position
        self.bullets.update()

        # delete the vanishing bullet
        for bullet in self.bullets.copy():
            if bullet.rect.bottom <= 0:
                self.bullets.remove(bullet)


    def _update_screen(self):
        """
        update the image on the screen
        switch to the new screen
        """
        # fill the screen at each run
        self.screen.fill(self.settings.bg_color) 
        self.ship.blitme()

        for bullet in self.bullets.sprites():
            bullet.draw_bullet()

        # visualisation
        # Update the full display Surface to the screen
        pygame.display.flip()







In [6]:
# if __name__ == 'main':
#     ai = AlienInvasion()
#     ai.run_game()

ai = AlienInvasion()
ai.run_game()

SystemExit: 